# scrape leafly for most reviewed strains

In [1]:
from bs4 import BeautifulSoup
import requests

import re

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
import os

chromedriver = "/home/nate/Bench/chromedriver"
os.environ["webdriver.chrome.driver"] = chromedriver

import pandas as pd

## soup

use beautifulsoup to grab html

use regex to extract links, ignore special promotions

In [2]:
url = 'https://www.leafly.com/explore'

page = requests.get(url).text    

soup = BeautifulSoup(page, "lxml")

In [3]:
a = str(soup.find_all('a'))

links = re.findall('<a class="ga_Explore_Strain_Tile" href="(.*?)">', a)

links = links[8:]

## selenium

use selenium to:

    1) go to each strain's page
    2) find number of reviews, use for number of pages (reviews/8)
    3) loop through all review pages
        3.1) click on 'read full review' for each review
        3.2) scrape review text

In [4]:
def n_pages(strain_link):
    
    '''
    this function executes step 2 from above
    '''
    
    driver.get('https://www.leafly.com' + strain_link)
    
    num_reviews = driver.find_element_by_xpath('//*[@id="main"]/div/section/div[2]/div[1]/div/div/ul/li[3]/a/span').text
    num_reviews = int(num_reviews[1:-1])
    num_pages = int(num_reviews/8) + 1
    
    return num_pages

In [5]:
def strain_scrape(strain_link, num_pages):
    
    '''
    this function executes step 3 from above
    '''
    
    reviews = []
    
    for page in range(num_pages):
        time.sleep(5)
        driver.get('https://www.leafly.com' + strain_link + '/reviews?page={}&sort=rating'.format(page))
        
        for review in range(1,9):
            time.sleep(1)
            try:
                driver.find_element_by_xpath('//*[@id="main"]/div/section/div[2]/div[2]/ul/li[{}]/div/div[2]/div[2]/div[1]/div/a'.format(review)).click()
                review_text = driver.find_element_by_xpath('//*[@id="main"]/div/section/div[2]/div[2]/div[4]/div[1]').text
                reviews.append(review_text)
                driver.execute_script("window.history.go(-1)")
            except:
                try:
                    driver.execute_script("window.history.go(-1)")
                    driver.execute_script("window.history.go(+1)")
                    driver.find_element_by_xpath('//*[@id="main"]/div/section/div[2]/div[2]/ul/li[{}]/div/div[2]/div[2]/div[1]/div/a'.format(review)).click()
                    review_text = driver.find_element_by_xpath('//*[@id="main"]/div/section/div[2]/div[2]/div[4]/div[1]').text
                    reviews.append(review_text)
                    driver.execute_script("window.history.go(-1)")
                except:
                    pass

    return reviews

## all the magic

booting up the website is a bit annoying, one of two agreement windows will pop up

then an email signup thing pops up after < 60 seconds

warning: this can take a very long time depending on number of reviews

In [37]:
driver = webdriver.Chrome(chromedriver)
driver.get('https://www.leafly.com')
try:
    driver.find_element_by_xpath('/html/body/div[6]/div/div/div/form/div[1]/div[1]/fieldset/label').click()
    driver.find_element_by_xpath('/html/body/div[6]/div/div/div/form/div[2]/button').click()
except:
    driver.find_element_by_xpath('/html/body/div[6]/div/div/form/fieldset/div/span[1]/label').click()
    driver.find_element_by_xpath('/html/body/div[6]/div/div/form/div/button').click()

# well this sucks, gotta kill an email popup
time.sleep(60)
driver.find_element_by_xpath('/html/body/div[9]/div/div/div/div[2]/form/div/a').click()

In [ ]:
all_reviews = []

for link in links:
    # find the number of pages to iterate through for each strain
    num_pages = n_pages(link)
    # iterate
    strain_reviews = strain_scrape(link, num_pages)
    # make a LoL for reviews
    all_reviews.append(strain_reviews)

In [ ]:
import pickle

pickle_out = open('raw_reviews.pkl', 'wb')
pickle.dump(all_reviews, pickle_out)
pickle_out.close()

## mvp scale review pull

time concerns... 

In [ ]:
strain_reviews = []

for page in range(977,1040):
    time.sleep(5)
    driver.get('https://www.leafly.com' + links[0] + '/reviews?page={}&sort=rating'.format(page))

    for review in range(1,9):
        time.sleep(1)
        try:
            driver.find_element_by_xpath('//*[@id="main"]/div/section/div[2]/div[2]/ul/li[{}]/div/div[2]/div[2]/div[1]/div/a'.format(review)).click()
            review_text = driver.find_element_by_xpath('//*[@id="main"]/div/section/div[2]/div[2]/div[4]/div[1]').text
            strain_reviews.append(review_text)
            driver.execute_script("window.history.go(-1)")
        except:
            try:
                time.sleep(5)
                driver.navigate().refresh()
                driver.find_element_by_xpath('//*[@id="main"]/div/section/div[2]/div[2]/ul/li[{}]/div/div[2]/div[2]/div[1]/div/a'.format(review)).click()
                review_text = driver.find_element_by_xpath('//*[@id="main"]/div/section/div[2]/div[2]/div[4]/div[1]').text
                strain_reviews.append(review_text)
                driver.execute_script("window.history.go(-1)")
            except:
                try:
                    time.sleep(5)
                    driver.execute_script("window.history.go(-1)")
                    driver.execute_script("window.history.go(+1)")
                    driver.find_element_by_xpath('//*[@id="main"]/div/section/div[2]/div[2]/ul/li[{}]/div/div[2]/div[2]/div[1]/div/a'.format(review)).click()
                    review_text = driver.find_element_by_xpath('//*[@id="main"]/div/section/div[2]/div[2]/div[4]/div[1]').text
                    strain_reviews.append(review_text)
                    driver.execute_script("window.history.go(-1)")
                except:
                    pass

In [30]:
len(strain_reviews)

2170

In [35]:
stored_reviews = pd.read_pickle('/home/nate/ds/metis/class_work/projects/project_fletcher/raw_reviews.pkl')

In [36]:
len(stored_reviews)

7809

In [33]:
stored_reviews = stored_reviews + strain_reviews

In [34]:
import pickle

pickle_out = open('raw_reviews.pkl', 'wb')
pickle.dump(stored_reviews, pickle_out)
pickle_out.close()